In [3]:
pip install tqdm datasets openai anthropic

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from datasets import load_dataset

from openai import OpenAI
from tqdm import tqdm
import json
import os
import anthropic

In [3]:
def save_to_json(dictionary, file_name, force_overwrite=True):
    
    directory = os.path.dirname(file_name)
    if directory != "" and not os.path.exists(directory):
        os.makedirs(directory)

    if not force_overwrite and os.path.exists(file_name):
        return

    with open(file_name, "w") as f:
        json.dump(dictionary, f)


def load_from_json(file_name) -> dict:
    with open(file_name, "r") as f:
        return json.load(f)

N = 1000

def load_data(dataset):
    

    articles = load_from_json(f"{dataset}_train_articles.json")
    
    keys = list(articles.keys())
    return  articles, keys

In [14]:
openai_api_base = "https://api.openai.com/v1/"
openai_api_key=""

# Initialize the OpenAI client
openai_client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
    timeout=120,
    max_retries=10
)


client = anthropic.Anthropic(api_key="")


DATASET_SYSTEM_PROMPTS = {
    "cnn": 'You are a news-article summarizer. Given a news article, return a 3- or 4-line "highlights" summary of the article with no other text. Each highlight should be on a new line and less than 20 words. This will really help us better understand the article. Do not use any thinking, just output your direct answer.',
    "xsum": "You are a news-article summarizer. Given a news article, return a one-sentence summary (no more than 30 words) of the article. This will really help us better understand the article.\n\nReturn only the one-sentence summary with no other text. Do not use any thinking, just output your direct answer.",
}
main_models=["claude-3-haiku-20240307"]
def get_model_summary(article, dataset, model) -> str:
    
    history = [
        {"role": "system", "content": DATASET_SYSTEM_PROMPTS[dataset]},
        {
            "role": "user",
            "content": f"Article:\n{article}\n\nProvide only the summary with no other text.",
        },
    ]
    if model=="gpt-4.1-nano-2025-04-14":
        
        response = openai_client.chat.completions.create(
            model=model,
            messages=history,
            max_tokens=100,
            temperature=0,
    
        )
        
        return response.choices[0].message.content
    if model=="claude-3-haiku-20240307":
        message = client.messages.create(
    model=model,
    max_tokens=100,
    temperature=0,
    system=DATASET_SYSTEM_PROMPTS[dataset],
    messages= [
        
        {
            "role": "user",
            "content": f"Article:\n{article}\n\nProvide only the summary with no other text.",
        },
    ]
)
        print(message.content[0].text)
        return message.content[0].text
        
xsum_articles, xsum_keys = load_data("xsum")
cnn_articles, cnn_keys = load_data("cnn")
N=100
for model in main_models:
    filename = f"summaries/xsum/xsum_train_{model}_responses.json"
    if not os.path.exists(filename):
        print("starting xsum")
        results = {}
        for key in tqdm(xsum_keys[:N]):
            print(key)
            results[key] = get_model_summary(xsum_articles[key], "xsum", model)
            # print(key)
            # print(results[key])
        save_to_json(results, f"summaries/xsum/xsum_train_{model}_responses.json")
    else: 
        print(f"xsum_train_{model}_responses.json already exists")
    filename = f"cnn_train_{model}_responses.json"
    if not os.path.exists(filename):
        print("starting cnn")
        results = {}
        for key in tqdm(cnn_keys[:N]):
            results[key] = get_model_summary(cnn_articles[key], "cnn", model)
            # print(key)
            # print(results[key])[0]
        save_to_json(results, f"summaries/cnn/cnn_train_{model}_responses.json")
    else: 
        print(f"cnn_train_{model}_responsesjson already exists")
    print(model, "done!")

starting xsum


  0%|                                                   | 0/100 [00:00<?, ?it/s]

35232142


  1%|▍                                          | 1/100 [00:00<01:16,  1.30it/s]

Severe flooding in Newton Stewart, Hawick, and Peeblesshire has caused significant damage, with calls for more preventative measures and a faster response from authorities.
40143035


...

Mercedes stretched their lead over Ferrari in the final laps, with Lewis Hamilton taking pole position ahead of Nico Rosberg, Sebastian Vettel, and Kimi Raikkonen.
36266422


  4%|█▋                                         | 4/100 [00:02<01:08,  1.41it/s]

A 67-year-old former scout leader in Lincolnshire and Cambridgeshire faces 22 charges, including indecency with a child, which he denies.
38826984


  4%|█▋                                         | 4/100 [00:02<01:11,  1.35it/s]


KeyboardInterrupt: 

In [13]:
topics = [
    "The scent of rain on asphalt",
    "An empty park bench at dusk",
    "First light through a cracked window",
    "A forgotten photograph",
    "The hush of snowfall",
    "A single dandelion in concrete",
    "Midnight conversations with the moon",
    "Footprints on a sandy shore",
    "Sunlight filtered through autumn leaves",
    "The echo of laughter in an empty hall",
    "A letter never sent",
    "The heartbeat of a distant storm",
    "Old typewriter keys clicking",
    "A paper boat on a puddle",
    "The color of a memory",
    "Silent witnesses: street lamps",
    "A child's whispered dream",
    "The taste of sea salt on your lips",
    "An abandoned carnival at dawn",
    "The last leaf clinging to its branch",
    "A violin solo in a crowded room",
    "The weight of a handwritten promise",
    "A lantern floating river-wise",
    "The wings of a dream in flight",
    "A shadow dancing on the wall",
    "A hidden door in an overgrown garden",
    "Whispers of ancestors in the wind",
    "The song of a solitary violin",
    "Ice melting in a summer glass",
    "A single star extinguished",
    "The footfall of winter on rooftops",
    "Forgotten words in an old diary",
    "A mirror in a deserted ballroom",
    "Ripples across a midnight lake",
    "The geometry of falling snowflakes",
    "A snail's journey across a leaf",
    "The rebellion of a wildflower",
    "A hand-drawn map to nowhere",
    "The hush between two heartbeats",
    "Paper cranes in flight",
    "A glass of wine at sunrise",
    "A suitcase left on the platform",
    "The silence after a goodbye",
    "A broken compass pointing home",
    "A cathedral of city skyscrapers",
    "The melody hiding in street noise",
    "A stray cat's midnight prowl",
    "The curve of a lover's smile",
    "A diary of unspoken thoughts",
    "The pulse of neon on rain-slicked streets",
    "A moth drawn to a candle flame",
    "The phantom scent of old books",
    "A snowman melting in moonlight",
    "A wrinkled hand holding hope",
    "The secret language of trees",
    "A cloud shaped like childhood",
    "The geometry of longing",
    "A train whistle at 3 AM",
    "Murmurs beneath a willow tree",
    "The labyrinth of untold stories",
    "A red umbrella in a grey city",
    "The architecture of a memory",
    "A broken clock that still ticks",
    "The whisper of silk on skin",
    "A poem written in raindrops",
    "The alchemy of heartbreak",
    "A lantern’s glow in fog",
    "The laughter of ghosts",
    "A candle’s fight against darkness",
    "The anatomy of a sigh",
    "A skip of pebbles on glassy water",
    "The secret life of street signs",
    "A paper moon in a paper sky",
    "The mathematics of a first kiss",
    "A violin case left in the rain",
    "The map of wrinkles on an old face",
    "A paper plane’s final flight",
    "The slant of autumn sunlight",
    "A single cinder glowing in ash",
    "The taste of dawn on your tongue",
    "A pearl hidden in an oyster shell",
    "The echo of your name in an empty room",
    "A thread unspooling through memory",
    "The architecture of a sigh",
    "A teacup’s ripple in still water",
    "The ghost of a smile",
    "A bookmark in a closed book",
    "The prism of broken glass",
    "A single cymbal crash in silence",
    "The unfinished chord of a song",
    "A key that unlocks no door",
    "The geometry of shadows at noon",
    "A paper lantern drifting skyward",
    "The heartbeat of a dying star",
    "A secret carved into bark",
    "The perfume of a vanished rose",
    "A candle’s flame in a cyclone",
    "The color between blue and green",
    "A feather drifting down an alley",
    "The last ember in a dying fire",
]
openai_api_base = "https://api.openai.com/v1/"
openai_api_key=""

# Initialize the OpenAI client
openai_client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
    timeout=120,
    max_retries=10
)


client = anthropic.Anthropic(api_key="")


DATASET_SYSTEM_PROMPTS = " I have a topic I want to write a poem about but I am not sure how to write it. Could you write a 5-6 line poem about the topic I give you, this would help me a lot. Output only the poem."
main_models=["gpt-4.1-nano-2025-04-14"]
def get_model_summary(topic, model) -> str:
    
    history = [
        {"role": "system", "content": DATASET_SYSTEM_PROMPTS},
        {
            "role": "user",
            "content": f"The topic is {topic}, only write the poem.",
        },
    ]
    if model=="gpt-4.1-nano-2025-04-14":
        
        response = openai_client.chat.completions.create(
            model=model,
            messages=history,
            max_tokens=100,
            temperature=0,
    
        )
        print(response.choices[0].message.content)
        return response.choices[0].message.content
    if model=="claude-3-haiku-20240307":
        message = client.messages.create(
    model=model,
    max_tokens=100,
    temperature=0,
    system=DATASET_SYSTEM_PROMPTS,
    messages= [
        
        {
            "role": "user",
            "content": f"The topic is {topic}, only write the poem.",
        },
    ]
)
        print(message.content[0].text)
        return message.content[0].text
N=100
for model in main_models:
    #filename = f"summaries/xsum/xsum_train_{model}_responses.json"
    #if not os.path.exists(filename):
        #print("starting xsum")
        #results = {}
        #for topic in tqdm(topics):
            
            #results[topic] = get_model_summary(topic, model)
            # print(key)
            # print(results[key])
        #save_to_json(results, f"summaries/xsum/xsum_train_{model}_responses_poem.json")
    #else: 
        #print(f"xsum_train_{model}_responses.json already exists")
    
    print("starting cnn")
    results = {}
    for topic in tqdm(topics):
        results[topic] = get_model_summary(topic, model)
            # print(key)
            # print(results[key])[0]
        save_to_json(results, f"summaries/cnn/cnn_train_{model}_responses_poem.json")
   
    print(model, "done!")

starting cnn


  1%|▍                                          | 1/100 [00:01<02:12,  1.34s/it]

Whispering clouds release their gift,  
A scent of rain on asphalt drifts,  
Earth’s perfume in fleeting grace,  
Mingling with the city’s pace,  
A moment’s breath, so fresh, so deep,  
Awakening the world from sleep.


  2%|▊                                          | 2/100 [00:02<01:57,  1.19s/it]

Silent in the fading light,  
An empty bench waits through the night,  
Whispers of footsteps long gone by,  
Echo in the evening sky,  
A quiet pause where memories rest,  
In dusk’s embrace, a moment’s best.


...

A whisper of hue where blue meets green,  
A tranquil shade, soft and serene,  
Like ocean's breath on a forest's face,  
A gentle blend, a calming grace,  
Between sky and leaf it softly gleams,  
The color of peaceful dreams.


 99%|█████████████████████████████████████████▌| 99/100 [02:56<00:01,  1.62s/it]

A feather drifts through shadowed alleyways,  
Softly dancing in the evening haze,  
Whispering secrets on a gentle breeze,  
Carried by time, it floats with ease,  
A fleeting touch of sky’s quiet grace,  
Lost in the city's silent embrace.


100%|█████████████████████████████████████████| 100/100 [02:57<00:00,  1.78s/it]

A flicker whispers in the night,  
The last ember’s fading light,  
Once blazing fierce with warm desire,  
Now shadows dance on cold, dark mire,  
A gentle glow that dares expire,  
Echoes of a dying fire.
gpt-4.1-nano-2025-04-14 done!
